In [ ]:
import numpy as np
import math
import plotly.express as px
import pandas as pd
import pickle
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import scipy.ndimage as ndimage
from tqdm import trange, tqdm
import plotly.graph_objects as go

In [ ]:
import generate_data
import algorithm
from utils import *
from fakekv import *

#from plots import *

### 设置参数

In [ ]:
sigma=0.1
n_eigenvectors=50
n_factors=2
eig_crit=delta=0.5
sim_crit=gamma=0.85
K=0

### 生成数据

In [ ]:
n_samples = 100
x_stretch = 20
y_stretch = 0
L = 108
var = 10000

In [ ]:
mol = FakeKV()
image_data = np.zeros((n_samples, L, L))
raw_data = np.zeros((n_samples, 3))
for i in trange(n_samples):
    angle = 90 * np.random.random()
    shift_x = x_stretch * (2 * np.random.random() - 1)
    shift_y = y_stretch * (2 * np.random.random() - 1)
    vol = mol.generate(angle=angle, shift=(shift_x, shift_y))
    vol = ndimage.rotate(vol, -30, (0,2), reshape=False, order=1)
    projz = np.sum(vol, axis=2, keepdims=False)
    gauss = np.random.normal(0, var**0.5, (L,L))
    noisy = projz + gauss
    image_data[i,:,:] = noisy
    raw_data[i,:] = [shift_x, shift_y, angle]

In [ ]:
vol = mol.generate(angle=0, shift=(0, 0))
X, Y, Z = np.mgrid[0:L, 0:L, 0:L]
fig = go.Figure(data=go.Volume(
    x=X.flatten(),
    y=Y.flatten(),
    z=Z.flatten(),
    value=vol.flatten(),
    isomin=1.0,
    isomax=20.0,
    opacity=0.1, # needs to be small to see through all surfaces
    surface_count=3, # needs to be a large number for good volume rendering
    width
    ))

In [ ]:
fig.update_layout(width=600, height=600)
fig.show()

In [ ]:
image_data.shape

In [ ]:
plt.imshow(image_data[0])
plt.show()

### 预处理

In [ ]:
image_data_ = np.reshape(image_data, (n_samples, -1))
print(image_data_.shape)

pca = PCA(n_components=4)
image_data_ = pca.fit_transform(image_data_)

scaler = StandardScaler()
image_data_ = scaler.fit_transform(image_data_)

print(image_data_.shape)

data = image_data_

### 距离矩阵

In [ ]:
W = calc_W(data, sigma)
W.shape

### 特征向量和特征值

In [ ]:
phi, Sigma = calc_vars(data, W, sigma, n_eigenvectors)
phi.shape, Sigma.shape

In [ ]:
best_matches, best_sims, all_sims = find_combos(phi, Sigma, n_factors, eig_crit, sim_crit)

In [ ]:
print(best_matches)
print(best_sims)

### 分解

In [ ]:
labels, C = split_eigenvectors(best_matches, best_sims, n_eigenvectors, K, n_factors)
labels.shape, C.shape

In [ ]:
manifolds = []
for m in range(n_factors):
    manifold = labels[0][np.where(labels[1]==m)[0]]
    manifolds.append(manifold)
for idx,m in enumerate(manifolds):
    if 1 in m:
        m1 = manifolds.pop(idx)
        manifolds.insert(0, m1)
manifolds

### 画图

In [ ]:
df = pd.DataFrame(np.hstack([data,phi]), 
                  columns=['x1','x2','x3','x4']+['%d-th eigen'%i for i in range(phi.shape[1])])
i=1
print('lambda =',Sigma[i])
fig = px.scatter_3d(df, x='x1', y='x2', z='x3', color='%d-th eigen'%i,
                    width=600, height=400)
fig.update_traces(marker_size = 1)
fig

In [ ]:
i = manifolds[0][0]
j = manifolds[0][1]
df = pd.DataFrame(phi, columns=['%d-th eigen'%i for i in range(phi.shape[1])])
fig = px.scatter(df, x='%d-th eigen'%i, y='%d-th eigen'%j, 
                 width=600, height=400)
fig

In [ ]:
i = manifolds[1][0]
j = manifolds[1][1]
df = pd.DataFrame(phi, columns=['%d-th eigen'%i for i in range(phi.shape[1])])
fig = px.scatter(df, x='%d-th eigen'%i, y='%d-th eigen'%j, 
                 width=600, height=400)
fig